# Match BGBM Collectors to Wikidata Items Using *Nearest Neighbour*, `eventDate` Involved

In this example we add `eventDate` of the source data, when the sample/occurrence was collected, to have a time reference, when the collector should have been  alive. 

Basically we …

- match of `canonical_string` of WikiData to `canonical_string` of the source collectors (abbreviated names and full names, if given), and
- parse collector source names beforehand to get individual names out of name lists in the source data, we have used <https://libraries.io/rubygems/dwc_agent>, and in general we
- follow the example of Niels Klazenga <https://github.com/nielsklazenga/avh-collectors/blob/master/match_names_to_wikidata_items.ipynb>
- calculate the name distance and also a similarity score for the matching of life span with the (sampling) `eventDate`, and finally we
- write the output to provide a DarwinCore attribution structure (for `verbatimName` we would need the `source_data` name(s))

Technical Notes — Review Code perhaps:

- TODO review score calculation of the matching of relating eventData with range of yob, yod
- TODO review DwC agent output, keep at this time custom columns for filter-sort-evaluation convenience
- (NN ⇌ Cosine) refactor relation: wd_matchtest ⇌ wikidata_unique (replaced wd_matchtest → wikidata_unique)


### Load Wikidata Data Set

Use Jupyter Notebook [create_wikidata_datasets_botanists-altlabel.ipynb](./create_wikidata_datasets_botanists-altlabel.ipynb) to generate matching data of botanists.

Now load the data and make them unique …

In [1]:
import pandas as pd
import pprint, time, os

explain_and_show_the_data = True
this_timestamp_for_data=20260210
# this_timestamp_for_data=time.strftime('%Y%m%d')

wikidata = pd.read_csv(
    # "data/wikidata_persons_botanists_20231030_1539.csv", # inverse match: [particle +] family, given
    # "data/wikidata_persons_botanists_20231116.csv",        # match: given [+ particle] + family[+ , suffix]
    # "data/wikidata_persons_botanists_20240312.csv", # with itemLabel + altLabel wyb, wye removed
    "data/wikidata_persons_botanists_20260210.csv",
    index_col=0, low_memory=False,
    dtype={
        'yob':'Int32',
        'yod':'Int32',
        'wyb':'Int32',
        'wye':'Int32'
    }    
)
# # # # # # # # # # # # # # # # 
# def convert_to_datetime(year):
#     year = int(year)
#     days = int((year - int(year)) * 365.25)
#     base_date = pd.to_datetime(f'{year}-01-01', errors="ignore")
#     # ,format='%Y-%m-%d', errors='coerce'
#     # base_date = pd.Period(year, freq='D')
#     return base_date + pd.DateOffset(days=days)
# 
# def convert_to_datetime(year):
#     base_date = pd.to_datetime(f'{year}-01-01', errors="ignore")
# print(wikidata.dtypes)

# wikidata['yob_converted'] = wikidata['yob'].apply(convert_to_datetime)
# wikidata['yod_converted'] = wikidata['yod'].apply(lambda x: pd.Period(x, freq='Y')) # Given date string "-286" not likely a datetime
# # # # # # # # # # # # # # # # 

if explain_and_show_the_data:
    pprint.pprint(wikidata.columns)
    display(wikidata.head())

Index(['item', 'itemLabel', 'surname', 'initials', 'canonical_string',
       'canonical_string_fullname', 'orcid', 'viaf', 'isni', 'harv', 'ipni',
       'abbr', 'bionomia_id', 'yob', 'yod', 'wikidata_link', 'orcid_link',
       'harv_link', 'ipni_link', 'bionomia_link'],
      dtype='str')


,item,itemLabel,surname,initials,canonical_string,canonical_string_fullname,orcid,viaf,isni,harv,ipni,abbr,bionomia_id,yob,yod,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,http://www.wikidata.org/entity/Q100142069,Frida Eggens,NaN,NaN,Eggens,Eggens,NaN,NaN,NaN,NaN,20045232-1,Eggens,NaN,<NA>,<NA>,http://www.wikidata.org/wiki/Q100142069,NaN,NaN,https://www.ipni.org/a/20045232-1,NaN
1,http://www.wikidata.org/entity/Q100142069,Frida Eggens,Frida,F.,F. Eggens,Frida Eggens,NaN,NaN,NaN,NaN,20045232-1,Eggens,NaN,<NA>,<NA>,http://www.wikidata.org/wiki/Q100142069,NaN,NaN,https://www.ipni.org/a/20045232-1,NaN
2,http://www.wikidata.org/entity/Q100146795,Elizabeth Harrison,Elizabeth,E.,E. Harrison,Elizabeth Harrison,NaN,NaN,NaN,NaN,NaN,NaN,Q100146795,1792,1834,http://www.wikidata.org/wiki/Q100146795,NaN,NaN,NaN,https://bionomia.net/Q100146795
3,http://www.wikidata.org/entity/Q100146795,Elizabeth Harrison,NaN,NaN,Mrs A. H.,Mrs A. H.,NaN,NaN,NaN,NaN,NaN,NaN,Q100146795,1792,1834,http://www.wikidata.org/wiki/Q100146795,NaN,NaN,NaN,https://bionomia.net/Q100146795
4,http://www.wikidata.org/entity/Q100146795,Elizabeth Harrison,Mrs Arnold,M. A.,M. A. Harrison,Mrs Arnold Harrison,NaN,NaN,NaN,NaN,NaN,NaN,Q100146795,1792,1834,http://www.wikidata.org/wiki/Q100146795,NaN,NaN,NaN,https://bionomia.net/Q100146795


In [2]:
# compile data having only unique canonical strings
# group by canonical name/string, count douplicated names
wd_matchtest = wikidata.groupby('canonical_string').agg({'item': ['count']}).reset_index()
wd_matchtest_fullnames = wikidata.groupby('canonical_string_fullname').agg({'item': ['count']}).reset_index()

if explain_and_show_the_data:
    display(wd_matchtest)
    display(wd_matchtest_fullnames)

,canonical_string,item
,,count
0,"""F."" Ryser",1
1,"""N.A. Antipova"" (lapsus)",1
2,"""N.A.Antipova"" (lapsus)",1
3,"""The grandmother of female scientists in Ghana""",1
4,"""Н. А. Антипова"" (lapsus)",1
...,...,...
171443,赵云鹏,1
171444,郭亚龙,1
171445,金井弘夫(Hiroo Kanai),1


,canonical_string_fullname,item
,,count
0,"""Fritz"" Ryser",1
1,"""N.A. Antipova"" (lapsus)",1
2,"""N.A.Antipova"" (lapsus)",1
3,"""The grandmother of female scientists in Ghana""",1
4,"""Н. А. Антипова"" (lapsus)",1
...,...,...
204788,赵云鹏,1
204789,郭亚龙,1
204790,金井弘夫(Hiroo Kanai),1


### Load Collectors Data Set

Data sources:

- option 1: Jupyter Notebook for [create_bgbm_gbif-occurrence_collectors_eventDate_dataset.ipynb](./create_bgbm_gbif-occurrence_collectors_eventDate_dataset.ipynb)
- option 2: Jupyter Notebook for `create_bgbm_botanypilot_collectors_dataset.ipynb` from SPARQL (not in this official documentation yet)

Then parse collector names to get single, separate collector names using `dwcagent`, use ruby gem package available at  <https://rubygems.org/gems/dwc_agent>:

- see [./bin/README.md](bin/README.md) to use ruby script `./bin/agent_parse4tsv.rb` for parsing text lines like `"Abbe,L.B., Abbe,E.C., Smitinand,T. & Rollet,B."`


In [3]:
# unique names parsed already by ruby gem package: dwcagent

# collectors = pd.read_csv("data/bgbm_collectors_20230510_1429_single-line_parsed_unique_names.tab", sep="\t")
collectors = pd.read_csv("data/VHde_doi-10.15468-dl.tued2e/occurrence_recordedBy_eventDate_occurrenceIDs_20230823_parsed.tsv", sep="\t", low_memory=False)

collectors.dropna(subset=['family'], inplace=True) # remove where family was NA, e.g. from originally «??» aso.

print("modify time using pd.Periode(…) to get it work also on very old dates...")
for col in ['eventDate_mean', 'eventDate_min', 'eventDate_max']:
    print("- convert", col, "to pd.Period(...) in collectors")
    collectors[col] = collectors[col].apply(
        lambda x: pd.Period(
            x, freq='s' if col.lower().endswith('mean') else 'D' # Seconds or Day level
        )
    ) # D=day level
    # see https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-period-aliases
print("modifing done.")

collectors.sort_values(by=['family', 'given','occurrenceID_firstsample'], inplace=True)
if explain_and_show_the_data: display(collectors)

modify time using pd.Periode(…) to get it work also on very old dates...
- convert eventDate_mean to pd.Period(...) in collectors
- convert eventDate_min to pd.Period(...) in collectors
- convert eventDate_max to pd.Period(...) in collectors
modifing done.


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
972,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Angeludis C., A.",parsed:C. Angeludis<SEP>A,cleaned:C. Angeludis<SEP>A,1,http://id.snsb.info/snsb/collection/462869/564...,2000-05-20 00:00:00,2000-05-20,2000-05-20
14972,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Gambia, A.",parsed:A,cleaned:A,6,http://id.snsb.info/snsb/collection/668020/726...,2008-03-29 08:00:00,2008-03-29,2008-03-30
32210,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nobs, Malcolm,A. & Galen Smith,S.",parsed:Malcolm Nobs<SEP>A<SEP>S. Galen Smith,cleaned:Malcolm Nobs<SEP>A<SEP>S. Galen Smith,1,https://herbarium.bgbm.org/object/B100089760,1949-08-31 00:00:00,1949-08-31,1949-08-31
30841,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Monteagudo A.; Peña; A.; Francis; R. & Quintuy...,parsed:A. Monteagudo<SEP>Peña<SEP>A<SEP>Franci...,cleaned:A. Monteagudo<SEP>Peña<SEP>A<SEP>Franc...,1,https://herbarium.bgbm.org/object/B100575848,2005-08-13 00:00:00,2005-08-13,2005-08-13
39755,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Sagastegui A.,A.",parsed:A. Sagastegui<SEP>A,cleaned:A. Sagastegui<SEP>A,1,https://herbarium.bgbm.org/object/B100711946,1989-08-29 00:00:00,1989-08-29,1989-08-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54150,Żelany,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Żelany,J.",parsed:J. Żelany,cleaned:J. Żelany,1,https://herbarium.bgbm.org/object/B100220196,2004-06-28 00:00:00,2004-06-28,2004-06-28
54151,Żelazny,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Żelazny,J.",parsed:J. Żelazny,cleaned:J. Żelazny,4,https://herbarium.bgbm.org/object/B100344466,2004-07-18 12:00:00,2004-06-09,2004-09-02
54152,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Ždanova,O.",parsed:O. Ždanova,cleaned:O. Ždanova,5,https://herbarium.bgbm.org/object/B100263330,1988-06-17 14:24:00,1986-08-18,1989-09-10
27617,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Lomonosova,M., Ždanova,O. & Šaulo,D.",parsed:M. Lomonosova<SEP>O. Ždanova<SEP>D. Šaulo,cleaned:M. Lomonosova<SEP>O. Ždanova<SEP>D. Šaulo,1,https://herbarium.bgbm.org/object/B100263331,1989-09-03 00:00:00,1989-09-03,1989-09-03


#### Check Composition of Parsed Collector Data

In [4]:
# TODO review code of abbreviated names and full name matching
if explain_and_show_the_data: 
    criterion_fullnames = collectors.given.str.contains('^\\w{3,}', na=False)
    print("Show collecors given name has (propably) a full name (%s records) …" % len(collectors[criterion_fullnames].index))
    display(collectors[criterion_fullnames])

Show collecors given name has (propably) a full name (598 records) …


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
2009,Abdallah,Raffael,NaN,NaN,NaN,NaN,NaN,NaN,"Bally,P.R.O., Abdallah, Raffael & Reichstein, T.",parsed:P.R.O. Bally<SEP>Raffael Abdallah<SEP>T...,cleaned:P.R.O. Bally<SEP>Raffael Abdallah<SEP>...,1,https://herbarium.bgbm.org/object/B200125981,1969-11-18 00:00:00,1969-11-18,1969-11-18
22502,Abdul,Kadir Bin,NaN,NaN,NaN,NaN,NaN,NaN,Kadir Bin Abdul,parsed:Kadir Bin Abdul,cleaned:Kadir Bin Abdul,1,https://herbarium.bgbm.org/object/B100184021,1950-03-20 00:00:00,1950-03-20,1950-03-20
17212,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN,Guilherme de Abreu (no. 103),parsed:Guilherme de Abreu,cleaned:Guilherme de Abreu,1,http://id.snsb.info/snsb/collection/22086/3086...,1938-08-24 00:00:00,1938-08-24,1938-08-24
15064,Adá,García,NaN,NaN,NaN,NaN,NaN,NaN,"García Adá, Luceño, Rico,E., Romero,T. & Varga...",parsed:García Adá<SEP>Rico Luceño<SEP>E. Romer...,cleaned:García Adá<SEP>Rico Luceño<SEP>E. Rome...,1,https://herbarium.bgbm.org/object/B100296455,1987-06-19 00:00:00,1987-06-19,1987-06-19
396,Ahagen,Schiers,NaN,NaN,NaN,NaN,NaN,NaN,"Ahagen, Schiers,C. & al.",parsed:Schiers Ahagen<SEP>C,cleaned:Schiers Ahagen<SEP>,1,https://herbarium.bgbm.org/object/B100194646,1987-07-03 00:00:00,1987-07-03,1987-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53432,Zernig,Kurt,NaN,NaN,NaN,NaN,NaN,NaN,"Zernig,Kurt",parsed:Kurt Zernig,cleaned:Kurt Zernig,3,https://herbarium.bgbm.org/object/B101139614,2019-06-29 08:00:00,2019-05-07,2019-09-28
23430,Zhuo,Zhou,NaN,NaN,NaN,NaN,NaN,NaN,"Kilian,N., Raab-Straube,E. von, Zhang Jianwen,...",parsed:N. Kilian<SEP>E.von Raab-Straube<SEP>Zh...,cleaned:N. Kilian<SEP>E. von Raab-Straube<SEP>...,63,https://herbarium.bgbm.org/object/B100517202,2012-08-26 04:57:08,2012-08-17,2012-09-02
53466,Zickendrath,Ernst,NaN,NaN,NaN,NaN,NaN,NaN,"Zickendrath,Ernst",parsed:Ernst Zickendrath,cleaned:Ernst Zickendrath,3,https://je.jacq.org/JE04006629,1896-11-27 08:00:00,1894-04-03,1898-09-20
53467,Zickendrath,Ernst,NaN,NaN,NaN,NaN,NaN,NaN,"Zickendrath,Ernst & Heyden,K.L.",parsed:Ernst Zickendrath<SEP>K.L. Heyden,cleaned:Ernst Zickendrath<SEP>K.L. Heyden,1,https://je.jacq.org/JE04007139,1900-09-01 00:00:00,1900-09-01,1900-09-01


In [5]:
# check the name-parsed columns if they are empty or need to be considerd as data for matching or not
if explain_and_show_the_data: 
    for parsed_name_part in ["particle", "suffix", "dropping_particle", "appellation"]:
        test_collectors = collectors.loc[(collectors[parsed_name_part].isna() == False)]
        print("\n----------------------------------------\nshow names with **%s** found %s records:\n" % (parsed_name_part, len(test_collectors.index)))
        display(test_collectors.head().get(["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title"]))    


----------------------------------------
show names with **particle** found 599 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
17212,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN
345,Aghababyan,M.,NaN,von,NaN,NaN,NaN,NaN
3461,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN
49459,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN
13591,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN



----------------------------------------
show names with **suffix** found 14 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
13941,August,Friedrich,II.,NaN,NaN,NaN,NaN,NaN
47752,Dogma,I.J.,Jr.,NaN,NaN,NaN,NaN,NaN
13727,Forsyth,W.,jr.,NaN,NaN,NaN,NaN,NaN
764,Grear,J.W.,Jr.,NaN,NaN,NaN,NaN,NaN
21713,Grear,J.W.,Jr.,NaN,NaN,NaN,NaN,NaN



----------------------------------------
show names with **dropping_particle** found 0 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title



----------------------------------------
show names with **appellation** found 1 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
13792,Sennen,NaN,NaN,NaN,NaN,NaN,Fr,NaN


Compile and compose `canonical_string…` of the collector data that we will later match the WikiData names with:

In [6]:
if explain_and_show_the_data: print("combine parts of names similar to WikiData's given name labels")

# 1. Prepare canonical name with conditional spacing: add space/comma directly to the value …
given = collectors['given'].fillna('')
# .where(condition, value_if_true) ensures spaces only appear when data is present
part = collectors['particle'].where(collectors['particle'].isna(), " " + collectors['particle'])
fam  = collectors['family'].where(collectors['family'].isna(), " " + collectors['family'])
suff = collectors['suffix'].where(collectors['suffix'].isna(), ", " + collectors['suffix'])
# 2. Vectorized concatenation
collectors['canonical_string_collector_parsed'] = (
    given + part.fillna('') + fam.fillna('') + suff.fillna('')
).str.strip()
# Optional: Clean up multiple spaces if they exist in the source data
collectors['canonical_string_collector_parsed'] = (
    collectors['canonical_string_collector_parsed'].str.replace(r'\s+', ' ', regex=True)
)

criterion = collectors["particle"].str.contains("\\w+ \\w+", na=False)

if explain_and_show_the_data: 
    # display(collectors['canonical_string_collector_parsed'][criterion].head())
    display(collectors[['canonical_string_collector_parsed', 'particle']][criterion].drop_duplicates().head(10))

combine parts of names similar to WikiData's given name labels


,canonical_string_collector_parsed,particle
3461,M.L. Reyna de Aguilar,Reyna de
6,H. Abbas al Ani,Abbas al
39486,J. Francisco del Aquila,Francisco del
28921,A.L.M. Marcailhou d' Aymeric,Marcailhou d'
37661,P.R.O. Ritchie in Bally,Ritchie in
33608,H. Perrier de la Bathie,Perrier de la
33056,A.M.F.J. Palisot de Beauvois,Palisot de
49070,D. Van der Ben,Van der
53674,B. van den Boom,van den
53673,P. van den Boom,van den


In [7]:
# move canonical_string_collector_parsed after column title (title was the last of the parsing columns)
col = collectors.pop("canonical_string_collector_parsed")
collectors.insert(collectors.columns.get_loc('title') + 1, col.name, col)

these_columns=["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title", 'canonical_string_collector_parsed']

if 'source_data' in collectors.columns:
    these_columns.append("source_data")

display(collectors.tail().get(these_columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data
54150,Żelany,J.,NaN,NaN,NaN,NaN,NaN,NaN,J. Żelany,"Żelany,J."
54151,Żelazny,J.,NaN,NaN,NaN,NaN,NaN,NaN,J. Żelazny,"Żelazny,J."
54152,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,O. Ždanova,"Ždanova,O."
27617,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,O. Ždanova,"Lomonosova,M., Ždanova,O. & Šaulo,D."
54153,Žíla,V.,NaN,NaN,NaN,NaN,NaN,NaN,V. Žíla,"Žíla,V."


In [8]:
# group and aggregate data to have unique name rows only for the matching of names later on
collectors_unique=collectors.groupby(['canonical_string_collector_parsed']).agg(
    family=('family', lambda x: list(x)[0]),
    given=('given', lambda x: list(x)[0]),
    suffix=('suffix', lambda x: list(x)[0]),
    particle=('particle', lambda x: list(x)[0]),
    dropping_particle=('dropping_particle', lambda x: list(x)[0]),
    nick=('nick', lambda x: list(x)[0]),
    appellation=('appellation', lambda x: list(x)[0]),
    title=('title', lambda x: list(x)[0]),
    source_data=('source_data', lambda x: list(x)[0]),
    occurrenceID_collectors_count= ('occurrenceID_count', 'sum'), # use count function
    occurrenceID_collectors_firstsample=('occurrenceID_firstsample', lambda x: list(x)[0]), # custom function, to get the first entry
    collectors_eventDate_mean=('eventDate_mean', 'mean'),
    collectors_eventDate_min=('eventDate_min', 'min'),
    collectors_eventDate_max=('eventDate_max', 'max')
).reset_index()

# move canonical_string_collector_parsed after column title
col = collectors_unique.pop("canonical_string_collector_parsed")
collectors_unique.insert(collectors_unique.columns.get_loc('title') + 1, col.name, col)
collectors_unique.drop_duplicates(inplace=True)

if explain_and_show_the_data: display(collectors_unique)

# column naming perhaps more clear (because we condensed the data)?
# collectors=collectors.add_suffix('_namegrouped') \
#  if not any(col.endswith("_namegrouped") for col in list(collectors.columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max
0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,"Angeludis C., A.",23,http://id.snsb.info/snsb/collection/462869/564...,1987-06-10 11:19:23,1949-08-31,2008-03-30
1,Silveira,A. A. da,NaN,NaN,NaN,NaN,NaN,NaN,A. A. da Silveira,"Silveira,A.A. da",6,https://herbarium.bgbm.org/object/B200129406a,1898-01-10 16:00:00,1897-11-01,1898-06-01
2,Thouars,A. A. du,NaN,NaN,NaN,NaN,NaN,NaN,A. A. du Thouars,"Thouars,A. A.du Petit-",1,https://herbarium.bgbm.org/object/B100365313,1829-01-01 00:00:00,1829-01-01,1829-01-01
3,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Aaronsohn,"Aaronsohn,A.",1,https://herbarium.bgbm.org/object/B100379341,1908-06-20 00:00:00,1908-06-20,1908-06-20
4,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Abaouz,"Jury,S.L., Abaouz,A. ,Lafkih,M.Ait & Griffiths...",5,https://herbarium.bgbm.org/object/B100217620,1997-07-10 02:00:00,1997-07-05,1997-07-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18423,Čelakovský,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Čelakovský,Čelakovský,1,https://dr.jacq.org/DR049067,1870-07-09 00:00:00,1870-07-09,1870-07-09
18424,Karjagin,İ.,NaN,NaN,NaN,NaN,NaN,NaN,İ. Karjagin,"Karjagin, İ.",1,https://herbarium.bgbm.org/object/B100601585,1934-02-01 00:00:00,1934-02-01,1934-02-01
18425,Černeva,Ž.,NaN,NaN,NaN,NaN,NaN,NaN,Ž. Černeva,"Markova,M. & Černeva,Ž.",1,https://herbarium.bgbm.org/object/B100310826,1975-04-19 00:00:00,1975-04-19,1975-04-19
18426,Fedorova,В.,NaN,NaN,NaN,NaN,NaN,NaN,В. Fedorova,"Tikhomirov,V. [Тихомиров,В.], Fedorova,T. & St...",1,https://herbarium.bgbm.org/object/B100630485,1991-06-04 00:00:00,1991-06-04,1991-06-04


In [9]:
if explain_and_show_the_data:
    print("show collectors with highest occurrenceID_collectors_count")
    display(collectors_unique.sort_values(by=['occurrenceID_collectors_count', 'family', 'given'], ascending=[False, True, True]).head(10))

show collectors with highest occurrenceID_collectors_count


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max
4743,Willing,E.,NaN,NaN,NaN,NaN,NaN,NaN,E. Willing,"Willing,E. & Eisenblätter,R.",189913,https://herbarium.bgbm.org/object/B100145955,1989-07-16 05:50:08,1919-01-01,2021-07-15
15076,Willing,R.,NaN,NaN,NaN,NaN,NaN,NaN,R. Willing,"Willing,R. & Willing,R.",188081,https://herbarium.bgbm.org/object/B100074611,2000-04-30 01:59:08,1976-06-15,2021-07-15
13811,Hein,P.,NaN,NaN,NaN,NaN,NaN,NaN,P. Hein,"Kilian,N., Hein,P. & Oberprieler,C.",9371,https://herbarium.bgbm.org/object/B100113011,2001-12-21 14:33:35,1976-01-01,2020-09-04
13066,Kilian,N.,NaN,NaN,NaN,NaN,NaN,NaN,N. Kilian,"Hunger,S. & Kilian,N.",9054,https://herbarium.bgbm.org/object/B100003448,2000-10-07 10:07:04,1982-06-01,2018-06-02
3734,Rodríguez,D.,NaN,NaN,NaN,NaN,NaN,NaN,D. Rodríguez,"Rodríguez,D., Monterrosa,J., Hernández,A. & Ma...",5206,https://herbarium.bgbm.org/object/B100038970,2014-04-24 04:12:33,2003-05-15,2022-03-20
4676,Tempel,E.,NaN,NaN,NaN,NaN,NaN,NaN,E. Tempel,"Tempel,E.",4433,https://dr.jacq.org/DR073621,1978-05-30 15:21:33,1949-01-01,1997-03-09
7825,Haussknecht,H.K.,NaN,NaN,NaN,NaN,NaN,NaN,H.K. Haussknecht,"Haussknecht, H.K.",4051,http://id.snsb.info/snsb/collection/474055/583...,1869-09-04 19:30:36,1856-01-01,1938-01-01
6835,Schimper,G.W.,NaN,NaN,NaN,NaN,NaN,NaN,G.W. Schimper,"Schimper, G.W. (no. 880)",3806,http://id.snsb.info/snsb/collection/108223/167...,1839-04-24 02:35:52,1828-09-24,1938-08-19
16542,Dürbye,T.,NaN,NaN,NaN,NaN,NaN,NaN,T. Dürbye,"Dürbye,T. & Kendzia,M.",3437,https://herbarium.bgbm.org/object/B100080892,2004-12-12 04:30:24,1905-06-15,2020-10-15
3516,Benkert,D.,NaN,NaN,NaN,NaN,NaN,NaN,D. Benkert,"Müller,U. & Benkert,D.",3324,https://herbarium.bgbm.org/object/B100124026,1989-07-09 01:08:50,1885-09-17,2006-05-07


### Set Up the Text Analysis

See https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536 for deeper understanding.

The `ngrams` function is used as an analyzer in the text search later.

In [10]:
import re
import unicodedata
!pip install ftfy # amazing text cleaning for decode issues..
from ftfy import fix_text

# Regex for unwanted special characters
CHARS_TO_REMOVE_RX = re.compile(r"[)(.|\[\]{}'„“”\"‚‘’›‹»«]|[,-./]|\sBD")
MULTIPLE_SPACES_RX = re.compile(r' +')

def ngrams(string, n=3):
    """
    Construct ngram(s) of a given text
     
    @param string: the text string to perform the ngram splitting on 
    @param n: character length of the particular (split) result text each
    @return: string as ngram
    """
    if not string:
        return []

    # (fix encoding errors)
    string = fix_text(string)

    # Normalization (IMPORTANT for Unicode)
    # NFC ensures that characters such as ‘é’ are treated as a single character
    # rather than as a combination of ‘e’ + accent.
    string = unicodedata.normalize('NFC', string)
    string = string.lower()    
    # (other languages often use different separators; kept generic here)
    string = string.replace('&', 'and').replace(',', ' ').replace('-', ' ')    
    # character cleanup but leave then as spaces
    string = CHARS_TO_REMOVE_RX.sub(' ', string)    
    # Normalization of spaces & case: ….title() also works with umlauts (e.g., “öllegard” -> “Öllegard”)
    string = string.title()
    string = MULTIPLE_SPACES_RX.sub(' ', string).strip()    
    # Padding & N-gram generation
    string = f" {string} "
    return [string[i : i + n] for i in range(len(string) - n + 1)]

In [11]:
import numpy as np
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

def calculateTFIDFmatchingOfData(query_data, match_data, n_neighbors=1):
    """
    Calculate a TF-IDF (Term Frequency — Inverse Document Frequency) matching with getNearestN

    @param query_data: DataFrame usually a pandas data column to query names or strings for
    @param match_data: DataFrame against to match with
    @param n_neighbors: number of similar data neighbors to calculate for (output as well; 5 = 5 times more output data, therefore default: 1)

    @requires NearestNeighbors()
    @requires getNearestNeighbour()
    @requires ngrams()
    @requires TfidfVectorizer()
    @requires NearestNeighbors()

    @return: DataFrame a data frame of matches with columns 'namematch_source_data', 'namematch_resource_data', 'namematch_distance'
    """
    # TODO if n_neighbors > 1 then output data need more explaining columns, 
    # or ensure the right ordering, because as e.g. n_neighbors=5  
    # then 5 sub-samples of the same name shall be exist
    
    start = time.time()
    
    # Unique values for performance, but directly as a list for stable index
    query_list = list(set(query_data))
    match_values = match_data.values # Cache für schnellen Zugriff
    
    print(f'Vectorizing {len(match_data)} reference items...')
    # Tip: ‘ngrams’ must be defined globally or appear here as a lambda.
    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
    tfidf_match_matrix = vectorizer.fit_transform(match_data)
    
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, n_jobs=-1, metric='cosine').fit(tfidf_match_matrix)
    
    print(f'Transforming {len(query_list)} queries...')
    tfidf_query_matrix = vectorizer.transform(query_list)
    
    # Start search
    distances, indices = nbrs.kneighbors(tfidf_query_matrix)
    
    print('Building DataFrame...')
    
    # We repeat each entry in query_list n_neighbors times
    # so that the column lengths for the DataFrame match
    query_repeated = np.repeat(query_list, n_neighbors)
    # TODO SUBSAMPLES n_neighbors: query_ids = np.repeat(np.arange(len(query_list)), n_neighbors)
    
    # Ensure that match_values is a flat NumPy array
    # .ravel() is the counterpart to .flatten(), but works more reliably
    flat_indices = indices.ravel()
    
    # We access the values and ensure that we have a NumPy format.
    match_hits = np.array(match_values)[flat_indices]
    
    matches = pd.DataFrame({
        # TODO SUBSAMPLES n_neighbors: 'group_index_id': query_ids,
        'namematch_source_data': query_repeated,
        'namematch_resource_data': match_hits,
        'namematch_distance': distances.ravel(),
        # TODO SUBSAMPLES n_neighbors: 'namematch_index': flat_indices
    })
    
    # Optional: Rounding the distance
    matches['namematch_distance'] = matches['namematch_distance'].round(4)

    print(f'Done in {time.time() - start:.2f}s')
    return matches

In [28]:
# some example data
samples = [
    ("simple name", "Klazenga, N."),
    ("simple name", "金井弘夫(Hiroo Kanai)"),
    ("simple name", "Н. А. Антипова"),
    ("data from collectors", collectors_unique["canonical_string_collector_parsed"].at[1]),
    ("data from match-test", collectors_unique["canonical_string_collector_parsed"].at[1]),
    ("data from match-test, full names", wd_matchtest_fullnames['canonical_string_fullname'].at[0])
]

print("Show ngram examples:")
for label, name in samples:
    print(f"- {label} “{name}”:", ngrams(name))

# some example data
print('\n(WikiData’s) canonical_string = (constructed) canonical_string_fullname:')
# We only take the first 5 lines of both columns.
short_names = wd_matchtest['canonical_string'].head(5)
long_names = wd_matchtest_fullnames['canonical_string_fullname'].head(5)

for short, long in zip(short_names, long_names):
    print(f"- {short} = {long}")

Show ngram examples:
- simple name “Klazenga, N.”: [' Kl', 'Kla', 'laz', 'aze', 'zen', 'eng', 'nga', 'ga ', 'a N', ' N ']
- simple name “金井弘夫(Hiroo Kanai)”: [' 金井', '金井弘', '井弘夫', '弘夫 ', '夫 H', ' Hi', 'Hir', 'iro', 'roo', 'oo ', 'o K', ' Ka', 'Kan', 'ana', 'nai', 'ai ']
- simple name “Н. А. Антипова”: [' Н ', 'Н А', ' А ', 'А А', ' Ан', 'Ант', 'нти', 'тип', 'ипо', 'пов', 'ова', 'ва ']
- data from collectors “A. A. da Silveira”: [' A ', 'A A', ' A ', 'A D', ' Da', 'Da ', 'a S', ' Si', 'Sil', 'ilv', 'lve', 'vei', 'eir', 'ira', 'ra ']
- data from match-test “A. A. da Silveira”: [' A ', 'A A', ' A ', 'A D', ' Da', 'Da ', 'a S', ' Si', 'Sil', 'ilv', 'lve', 'vei', 'eir', 'ira', 'ra ']
- data from match-test, full names “"Fritz" Ryser”: [' Fr', 'Fri', 'rit', 'itz', 'tz ', 'z R', ' Ry', 'Rys', 'yse', 'ser', 'er ']

(WikiData’s) canonical_string = (constructed) canonical_string_fullname:
- "F." Ryser = "Fritz" Ryser
- "N.A. Antipova" (lapsus) = "N.A. Antipova" (lapsus)
- "N.A.Antipova" (lapsus) 

Vectorize Wikidata names. Background: We use an information retrieval technique (Term Frequency — Inverse Document Frequency, blog [towardsdatascience.com/tf-idf-explained…](https://towardsdatascience.com/tf-idf-explained-and-python-sklearn-implementation-b020c5e83275)) for matching the source names with WikiData names, for that a calculated dinsance measure of the name match will help to match similar names and distinguish names that are rather no match. In general see also https://scikit-learn.org, https://pypi.org/project/scikit-learn/.

### Perform the Matching

Perform the nearest neighbour (NN) matches on the (BGBM) collector names and create a data frame with matches, and we try to distinguish abbreviated and full names in the source to better match source data and WikiData ... (can take 5 to 10 minutes)

Now convert a collection of raw documents to a matrix of TF-IDF features and set up the function that performs the matches...

In [13]:
criterion_fullnames = collectors_unique.given.str.contains('^\\w{3,}', na=False)
collectors_names = collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values
# collectors_names = set(collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values)
print("Calculate rather the abbreviated names only …")
matches = calculateTFIDFmatchingOfData(collectors_names, wd_matchtest['canonical_string']) 

matches = matches.sort_values(['namematch_distance'])
matches = matches.reset_index(names=['old_index'])

if explain_and_show_the_data: display(matches)

Calculate rather the abbreviated names only …
Vectorizing 171448 reference items...
Transforming 17974 queries...
Building DataFrame...
Done in 89.43s


,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,5919,L. Dubois,L. Dubois,0.0000
1,5900,N. Wendt,N.Wendt,0.0000
2,5899,Mendonça,Mendonça,0.0000
3,5895,L.A. Braas,L. A. Braas,0.0000
4,5894,C.E. Moss,C. E. Moss,0.0000
...,...,...,...,...
17969,3789,Wyrwis,Lewis,0.7261
17970,5014,A.L. Mpandzou,M. MP,0.7272
17971,16247,M.C. Ejiofor,M.C.C.,0.7335
17972,3966,V. Avjetijezs,N. A. Mekhtijeva,0.7468


In [14]:
this_name = "Kotsch"
criterion = matches['namematch_source_data'].str.contains(this_name)
display(matches[criterion])

,old_index,namematch_source_data,namematch_resource_data,namematch_distance
1389,4290,T. Kotschy,T. Kotschy,0.0000
3746,7772,C.G.T. Kotschy,C. G. T. Kotschy,0.0000
7871,14787,K.G.T. Kotschy,K. G. T. Kotschy,0.0000
10854,8071,C.G.T. Kotschyi,C. G. T. Kotschy,0.1145
11125,2845,C.G. Kotschy,Kotschy,0.1238
12202,3805,Th Kotschy,Kotschy,0.1805


In [15]:
# criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_fullnames = collectors_unique['canonical_string_collector_parsed'][criterion_fullnames].values

print("Calculate rather the full names only …")
matches_fullnames = calculateTFIDFmatchingOfData(collectors_fullnames, wd_matchtest_fullnames['canonical_string_fullname']) 

matches_fullnames = matches_fullnames.sort_values(['namematch_distance'])
matches_fullnames = matches_fullnames.reset_index(names=['old_index'])

if explain_and_show_the_data: display(matches_fullnames)

Calculate rather the full names only …
Vectorizing 204793 reference items...
Transforming 454 queries...
Building DataFrame...
Done in 6.37s


,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,447,Theodor Magnus Fries,Theodor Magnus Fries,0.0000
1,446,David Heinrich Hoppe,David Heinrich Hoppe,0.0000
2,445,Johann Friedrich Prahl,Johann Friedrich Prahl,0.0000
3,444,Franz Kappel,Franz Kappel,0.0000
4,442,Thomas Pichler,Thomas Pichler,0.0000
...,...,...,...,...
449,60,Mukram Kipka,R.Seipka,0.6375
450,82,Not Indicated,De Not.,0.6382
451,5,NMà Retournemer,A. Letourneux,0.6470
452,282,Inst Bio-Ped-Psamm Xinjiang,Insam,0.6647


### Create Output Results

Combine the matches data frame back to the (BGBM) collectors and Wikidata items …

In [16]:
if explain_and_show_the_data: print("join matches data frame back to source collectors dataframe")
collectors_matches = pd.merge(
    collectors_unique, matches, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

if explain_and_show_the_data: display(collectors_matches.head())

join matches data frame back to source collectors dataframe


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,"Angeludis C., A.",23,http://id.snsb.info/snsb/collection/462869/564...,1987-06-10 11:19:23,1949-08-31,2008-03-30,17918,A,A. A.,0.2696
1,Silveira,A. A. da,NaN,NaN,NaN,NaN,NaN,NaN,A. A. da Silveira,"Silveira,A.A. da",6,https://herbarium.bgbm.org/object/B200129406a,1898-01-10 16:00:00,1897-11-01,1898-06-01,13686,A. A. da Silveira,A. A. d. Silveira,0.2001
2,Thouars,A. A. du,NaN,NaN,NaN,NaN,NaN,NaN,A. A. du Thouars,"Thouars,A. A.du Petit-",1,https://herbarium.bgbm.org/object/B100365313,1829-01-01 00:00:00,1829-01-01,1829-01-01,12695,A. A. du Thouars,A. A. D. Thouars,0.2364
3,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Aaronsohn,"Aaronsohn,A.",1,https://herbarium.bgbm.org/object/B100379341,1908-06-20 00:00:00,1908-06-20,1908-06-20,12961,A. Aaronsohn,A. Aaronsohn,0.0000
4,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Abaouz,"Jury,S.L., Abaouz,A. ,Lafkih,M.Ait & Griffiths...",5,https://herbarium.bgbm.org/object/B100217620,1997-07-10 02:00:00,1997-07-05,1997-07-14,16121,A. Abaouz,J.A.Abad,0.5876


In [17]:
if explain_and_show_the_data: print("show full name matches, and append them to all matches")
collectors_matches_fullname = pd.merge(
    collectors_unique, matches_fullnames, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

if explain_and_show_the_data: display(collectors_matches_fullname.head())

show full name matches, and append them to all matches


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,Sauze,Abbé,NaN,NaN,NaN,NaN,NaN,NaN,Abbé Sauze,"Sauze,Abbé,J.",1,https://herbarium.bgbm.org/object/B100263049,1893-06-01 00:00:00,1893-06-01,1893-06-01,378,Abbé Sauze,Abbé Fray,0.5195
1,Hassan,Abdisalam Sheikh,NaN,NaN,NaN,NaN,NaN,NaN,Abdisalam Sheikh Hassan,"Friis,I., Vollesen,K. & Abdisalam Sheikh Hassan",7,https://herbarium.bgbm.org/object/B100003700,1987-05-30 14:24:00,1987-05-27,1987-06-14,78,Abdisalam Sheikh Hassan,Abdisalam S. Hassan,0.2153
2,Hassan,Abdisalem Sheikh,NaN,NaN,NaN,NaN,NaN,NaN,Abdisalem Sheikh Hassan,"Friis,I, Vollesen,K., & Abdisalem Sheikh Hassan",1,https://herbarium.bgbm.org/object/B100003663,1987-06-07 00:00:00,1987-06-07,1987-06-07,426,Abdisalem Sheikh Hassan,Abdisalam S. Hassan,0.3453
3,Khalek,Abel,NaN,el,NaN,NaN,NaN,NaN,Abel el Khalek,Abel el Khalek,1,https://herbarium.bgbm.org/object/B100763849,1926-11-24 00:00:00,1926-11-24,1926-11-24,175,Abel el Khalek,Abel,0.4952
4,García,Abisaii,NaN,NaN,NaN,NaN,NaN,NaN,Abisaii García,"Miller,J.S. & García,Abisaii",1,https://herbarium.bgbm.org/object/B100375518,1987-06-08 00:00:00,1987-06-08,1987-06-08,408,Abisaii García,Abisai Josue García-Mendoza,0.4565


In [18]:
collectors_all_matches=pd.concat([collectors_matches, collectors_matches_fullname])
collectors_all_matches.sort_values(by=['namematch_distance', 'family'], ascending=[True, True], inplace=True)
if explain_and_show_the_data:
    print("show match results of all abbreviated and full names")
    display(collectors_all_matches.head())

show match results of all abbreviated and full names


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_distance
1074,A.A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A.A,"Magalhães G.,A.A.",3,https://herbarium.bgbm.org/object/B100247806,1895-04-23 08:00:00,1894-01-01,1896-07-25,6405,A.A,A. A.,0.0
3,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Aaronsohn,"Aaronsohn,A.",1,https://herbarium.bgbm.org/object/B100379341,1908-06-20 00:00:00,1908-06-20,1908-06-20,12961,A. Aaronsohn,A. Aaronsohn,0.0
4714,Abbe,E.C.,NaN,NaN,NaN,NaN,NaN,NaN,E.C. Abbe,"Raup,H.M. & Abbe,E.C.",2,https://herbarium.bgbm.org/object/B100241637,1946-05-22 12:00:00,1932-07-26,1960-03-18,2602,E.C. Abbe,E. C. Abbe,0.0
11011,Abbe,L.B.,NaN,NaN,NaN,NaN,NaN,NaN,L.B. Abbe,"Abbe,L.B., Abbe,E.C., Smitinand,T. & Rollet,B.",1,https://herbarium.bgbm.org/object/BGT0003826,1960-03-18 00:00:00,1960-03-18,1960-03-18,9206,L.B. Abbe,L. B. Abbe,0.0
3826,Abbiw,D.K.,NaN,NaN,NaN,NaN,NaN,NaN,D.K. Abbiw,"Abbiw,D.K.",1,https://herbarium.bgbm.org/object/B180010225,1984-06-01 00:00:00,1984-06-01,1984-06-01,2913,D.K. Abbiw,D. K. Abbiw,0.0


In [19]:
# criterion = collectors_all_matches['canonical_string_collector_parsed'].map(lambda x: x.startswith('Kotschy'))
# print("Show example of «Kotschy…» with namematch distances from 0.0 to 1.0 (in Cosine Similiarity we had 0.5 … 1.0)")
# collectors_all_matches[criterion]

In [20]:
# Save the plain name matching results only ...

do_custom_data_aggregation=False
if do_custom_data_aggregation:
    if not os.path.exists('data'):
        print("Make data directory for saving …")
        os.makedirs('data')
        
    this_output_file='data/results_bgbm_collectors-eventDate_vs_wikidata-botanists_kneighbor_plain-names_%s.csv' % (
        this_timestamp_for_data
    )
    
    collectors_all_matches.to_csv(this_output_file)
    
    print("Wrote plain name matches of collector names into %s (%d kB)" % 
        (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
    )

### Merge Matched Data and WikiData’s

Review (TODO)
- evaluate time references: `eventDate` ~ `yob`, `wyb`—perhaps define a score value that could integrate all scores from properties we need for decision of the name matching (name distance, eventDate ~ year of birth/work year begin aso.)
- merge abbreviated and full name data properly, distinguish abbrevited match and full name match
- refactor `collectors_matches` or `collectors_matches_g1` aso. to `collectors_all_matches`
- refactor `collectors` to `collectors_unique`
- refactor `matches`to `matches_abbr` or distinguish `matches_fullname`

Now
1. merge now the matching data and the wiki data’s on the conaonical string name
2. later aggregate fine tuned, checking if multiple same (canonical string) names relate to multiple different persons (we use wd-items (the Q1233242 thing), and wd-item-labels to aggregate on) … aso.
3. save those data tables

In [21]:
if explain_and_show_the_data: print("merge now the matching data and the wiki data’s on the conaonical string name")
    
collectors_matches_tmp_names_abbreviated = pd.merge(
    collectors_matches, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string'
)
collectors_matches_tmp_fullnames = pd.merge(
    collectors_matches_fullname, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string_fullname'
)
collectors_matches_g1_merged_wikidata = pd.concat(
    [collectors_matches_tmp_names_abbreviated, collectors_matches_tmp_fullnames]
    , ignore_index=True
)

merge now the matching data and the wiki data’s on the conaonical string name


In [22]:
if explain_and_show_the_data:
    # print(collectors_matches_g1_merged_wikidata.columns)
    this_name="Kotsch"
    print(f"Show example data of «…{this_name}…» with namematch distances")
    
    display(
        collectors_matches_g1_merged_wikidata[
            collectors_matches_g1_merged_wikidata['canonical_string_collector_parsed'].str.contains(this_name, na=False)
        ]
        .sort_values('namematch_distance', ascending=True)
        [[
            # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
            'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
            # 'source_data',
            'namematch_source_data', 'namematch_resource_data', 'namematch_distance', 
            'itemLabel', 
            'canonical_string_fullname', # canonical_string_fullname contains the former itemMatchingLabel
            'wikidata_link',
            'collectors_eventDate_min', 'collectors_eventDate_max',
            'yob', 'yod',
        ]]
    )

Show example data of «…Kotsch…» with namematch distances


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_distance,itemLabel,canonical_string_fullname,wikidata_link,collectors_eventDate_min,collectors_eventDate_max,yob,yod
4170,2215,http://id.snsb.info/snsb/collection/108230/167...,C.G.T. Kotschy,C. G. T. Kotschy,0.0000,Theodor Kotschy,C. G. T. Kotschy,http://www.wikidata.org/wiki/Q113299,1812-06-20,1900-08-01,1813,1866
4171,2215,http://id.snsb.info/snsb/collection/108230/167...,C.G.T. Kotschy,C. G. T. Kotschy,0.0000,Theodor Kotschy,Carl Georg Theodor Kotschy,http://www.wikidata.org/wiki/Q113299,1812-06-20,1900-08-01,1813,1866
13960,36,http://id.snsb.info/snsb/collection/16719/2549...,K.G.T. Kotschy,K. G. T. Kotschy,0.0000,Theodor Kotschy,Karl Georg Theodor Kotschy,http://www.wikidata.org/wiki/Q113299,1837-01-01,1842-01-07,1813,1866
13959,36,http://id.snsb.info/snsb/collection/16719/2549...,K.G.T. Kotschy,K. G. T. Kotschy,0.0000,Theodor Kotschy,K. G. T. Kotschy,http://www.wikidata.org/wiki/Q113299,1837-01-01,1842-01-07,1813,1866
13961,36,http://id.snsb.info/snsb/collection/16719/2549...,K.G.T. Kotschy,K. G. T. Kotschy,0.0000,Theodor Kotschy,Karl George Theodor Kotschy,http://www.wikidata.org/wiki/Q113299,1837-01-01,1842-01-07,1813,1866
23810,1,https://herbarium.bgbm.org/object/B101086119,Carl Georg Theodor Kotschy,Carl Georg Theodor Kotschy,0.0000,Theodor Kotschy,Carl Georg Theodor Kotschy,http://www.wikidata.org/wiki/Q113299,1853-09-22,1853-09-22,1813,1866
21435,294,http://id.snsb.info/snsb/collection/117808/176...,T. Kotschy,T. Kotschy,0.0000,Theodor Kotschy,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299,1836-01-01,1939-11-19,1813,1866
21434,294,http://id.snsb.info/snsb/collection/117808/176...,T. Kotschy,T. Kotschy,0.0000,Theodor Kotschy,T. Kotschy,http://www.wikidata.org/wiki/Q113299,1836-01-01,1939-11-19,1813,1866
4172,1,https://herbarium.bgbm.org/object/B100160086,C.G.T. Kotschyi,C. G. T. Kotschy,0.1145,Theodor Kotschy,C. G. T. Kotschy,http://www.wikidata.org/wiki/Q113299,1839-10-03,1839-10-03,1813,1866
4173,1,https://herbarium.bgbm.org/object/B100160086,C.G.T. Kotschyi,C. G. T. Kotschy,0.1145,Theodor Kotschy,Carl Georg Theodor Kotschy,http://www.wikidata.org/wiki/Q113299,1839-10-03,1839-10-03,1813,1866


## Write DarwinCore Attribution Output

Here we map table data fields to fields of DarwinCore Attribution (<https://github.com/tdwg/attribution/>, <https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml>) 

### Scoring

Individual scored properties should actually be balanced in such a way that one can simply add up these different property scores; in this case, assessment of the calculated values is still necessary. The problem here with calculation with a distance measure is that we have the opposite of similarity, whose distance can become greater than 1, which must somehow be mapped to a scope of 0 … 1 (or -1 … 0 … 1) (TODO review).

General thoughts: With a score of -1 to 1, it can be assumed that:
* -1 means full devaluation or no agreement
* 1 means full upvoting or agreement, and
* 0 can have several interpretations: it is in between, or no rating possible, or missing values.

### Task to Be Solved in Evaluating the Life Time ~ Rating/Scoring

We have grouped the collection date (evenDate) to the name in the source data, so it may be that for (abbreviated) names, e.g. “Bachmann, F.”, the collection date is valid for *several* personal names, not just one. This must be taken into account when considering and evaluating whether the life data match the collection date. The rating of the life data has the following idea:

| Score (life time) | Remarks | 
|--|--|
| 1.0  | complete match                     |
| 0.5  | somewhat correct, but has errors or mistakes, indicating multiple person names    |
| 0.0   | no evaluation (or not possible) |
| -0.5 | is rather to be rejected, indicating multiple person names and possibly overlapping time spans of the collection date of different person names, or mistakes in the original data |
| -1.0 | completely rejected                |

### Task to Be Solved With Several Names ~ Assessment/Score

Since we do not know if there are other possible names somewhere when there is only one name, we cannot assign a “1” (= full agreement) with certainty, so it was decided that if only 1 name was found, this would be evaluated as zero, in the sense of no evaluation. So when evaluating the multiple names, only the mismatches are evaluated, according to the idea:

| Score (multiple names) | Remarks | 
|--|--|
| 1.0  | this value (=full upvoting or agreement) would never be set in this regard, since we do not know all the full names of the cosmos ;-), and could state this score certainty of 1.0 |
| 0.0 | no evaluation, because only 1 name found | 
| less than 0 | multiple names found, i.e. deduction (perhaps just -0.5, as a decision needs to be made) | 

---

TODO review interpretation:

- the fields are defined in <https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml> and regarding from this DwC-attribution concept: is it correct to map it like the following (`name` would represent the *interpreted* resource name (in long format), not the *source* collector `name` (in (theoretically) long format))?
    ```
    name          ← itemLabel (wikiData)
    alternateName ← canonical_string_collector_parsed (actual collector name)
    ```

In [23]:
# TODO further evaluation or filtering, counting, clean up aso.
pprint.pprint(collectors_matches_g1_merged_wikidata.columns)

Index(['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick',
       'appellation', 'title', 'canonical_string_collector_parsed',
       'source_data', 'occurrenceID_collectors_count',
       'occurrenceID_collectors_firstsample', 'collectors_eventDate_mean',
       'collectors_eventDate_min', 'collectors_eventDate_max', 'old_index',
       'namematch_source_data', 'namematch_resource_data',
       'namematch_distance', 'item', 'itemLabel', 'surname', 'initials',
       'canonical_string', 'canonical_string_fullname', 'orcid', 'viaf',
       'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod',
       'wikidata_link', 'orcid_link', 'harv_link', 'ipni_link',
       'bionomia_link'],
      dtype='str')


In [24]:
# refactor namematch_similarity → namematch_distance
# refactor namematch_similarity_annotation → namematch_distance_annotation
# refactor custom_namematch_similarity → custom_namematch_namematch
# refactor sort_values
# refactor collectors_eventDate_mean
# refactor collectors_eventDate_min
# - refactor yob_is_lt_eventDate_min
# refactor collectors_eventDate_max
# - refactr yod_is_gt_eventDate_max
# refactor custom_score_lifetime            → custom_score_lifetime_data
# refactor custom_score_lifetime_annotation → custom_score_lifetime_data_annotation
# refactor namematch_similarity             → namematch_distance
# refactor namematch_similarity_annotation  → namematch_distance_annotation
# refactor custom_namematch_similarity → custom_namematch_namematch
# refactor sort_values
collectors_wikidata_cossimOrKmeans = collectors_matches_g1_merged_wikidata[
    [
        'canonical_string_collector_parsed', 'family', 'given', 
        'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
        'source_data',
        'namematch_source_data', 'namematch_resource_data', 'namematch_distance', 
        'item', 'canonical_string', 'itemLabel',
        'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 
        'collectors_eventDate_mean', 'collectors_eventDate_min', 'collectors_eventDate_max',
        'yob', 'yod',
        # 'wyb'
    ]
].copy()

# order by canonical_string_collector_parsed (actual collector name) (asc)
#   order by similarity (desc) or namematch_distance (asc), 
#     order by number of Wikidata items (asc) and 
#       order by number of collections (desc)
collectors_wikidata_cossimOrKmeans.sort_values(
    by=['canonical_string_collector_parsed', 'namematch_distance', 'family', 'given'], 
    ascending=[True, True, True, True], 
    inplace=True
)

dwcagent_attr_output=collectors_wikidata_cossimOrKmeans.get([
    "occurrenceID_collectors_firstsample", 
    "canonical_string_collector_parsed",
    'family', 'given',
    "namematch_distance", 
    "source_data", 
    "itemLabel", 
    "item",
    "collectors_eventDate_min",
    "collectors_eventDate_max",
    'yob', 'yod'
]).copy().drop_duplicates(ignore_index=True)


# dwcagent_attr_output['canonical_string_collector_parsed'].replace(to_replace=r'([^,]+),\s*(.+)', value='\\2 \\1', inplace=True, regex=True)
dwcagent_attr_output['canonical_string_collector_parsed'] = dwcagent_attr_output['canonical_string_collector_parsed'].astype(object)
dwcagent_attr_output['canonical_string_collector_parsed'] = dwcagent_attr_output['canonical_string_collector_parsed'].replace(
    to_replace=r'([^,]+),\s*(.+)',
    value=r'\\2 \\1',
    regex=True
)

dwcagent_attr_output['namematch_distance_annotation'] = dwcagent_attr_output['namematch_distance'].astype(str).str.replace(r'(.+)', '\\1 (k-means distance)', regex=True)
# dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'namematch_distance_annotation', '', allow_duplicates=True)

dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'life_time_periode', '', allow_duplicates=True)

combine_life_times = lambda this_df: ("%s-%s" % (this_df["yob"], this_df["yod"])).replace(r"<NA>", "?")
dwcagent_attr_output["life_time_periode"]=dwcagent_attr_output.apply(combine_life_times, axis="columns")

# dwcagent_attr_output["life_time_periode"]

years_from_birth_until_first_collection_activity = 10
dwcagent_attr_output["yob_is_lt_eventDate_min"] = dwcagent_attr_output["yob"] + years_from_birth_until_first_collection_activity < dwcagent_attr_output["collectors_eventDate_min"].dt.year
dwcagent_attr_output["yod_is_gt_eventDate_max"] = dwcagent_attr_output["yod"] > dwcagent_attr_output["collectors_eventDate_max"].dt.year
dwcagent_attr_output["custom_score_lifetime_data"] = 0.0
dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'custom_score_lifetime_data_annotation', '', allow_duplicates=True)

import numpy as np

# 1. Variables - Ensure they are converted to standard numpy booleans
# We fill NA with False for the logical check to satisfy numpy.select
y_to_activity = years_from_birth_until_first_collection_activity 

b_ok = dwcagent_attr_output["yob_is_lt_eventDate_min"].fillna(False).astype(bool)
d_ok = dwcagent_attr_output["yod_is_gt_eventDate_max"].fillna(False).astype(bool)

# To still detect the "NA" cases in your conditions, we need the original NA info:
b_isna = dwcagent_attr_output["yob_is_lt_eventDate_min"].isna()
d_isna = dwcagent_attr_output["yod_is_gt_eventDate_max"].isna()

# 2. Updated Conditions (using the sanitized booleans and the NA-checkers)
conditions = [
    (b_ok == True)  & (d_ok == True),   # Both match
    (b_ok == True)  & (d_isna),         # Birth matches, death unknown
    (b_isna)        & (d_ok == True),   # Birth unknown, death matches
    (b_isna)        & (d_isna),         # Both unknown
    (b_ok == True)  & (d_ok == False) & (~d_isna),  # Birth matches, death contradicts
    (b_ok == False) & (~b_isna) & (d_ok == True),   # Birth contradicts, death matches
    (b_ok == False) & (~b_isna) & (d_isna),         # Birth contradicts, death unknown
    (b_isna)        & (d_ok == False) & (~d_isna),  # Birth unknown, death contradicts
    (b_ok == False) & (~b_isna) & (d_ok == False) & (~d_isna) # Both contradict
]

scores = [1.0, 1.0, 1.0, 0.0, 0.5, 0.5, -0.5, -0.5, -1.0]

labels = [
    "full match",
    "OK? year of death is missing",
    "OK? year of birth is missing",
    "unknown life time",
    f"OK yob + {y_to_activity}, but yod not matching, check name and lifetime data",
    f"yob + {y_to_activity} not matching, OK yod, check name and lifetime data",
    f"yob + {y_to_activity} not matching, yod unknown, check name and lifetime data",
    f"yob unknown, yod not matching, check name and lifetime data",
    f"life time not matching any eventDate (yob + {y_to_activity} … yod)"
]

# 3. Apply Scoring and Annotations
dwcagent_attr_output["custom_score_lifetime_data"] = np.select(conditions, scores, default=0.0)
dwcagent_attr_output["custom_score_lifetime_data_annotation"] = np.select(conditions, labels, default="check data consistency")

# 4. Handle Ambiguity
is_duplicated = dwcagent_attr_output['canonical_string_collector_parsed'].duplicated(keep=False)
dwcagent_attr_output["custom_score_multiple_names"] = np.where(is_duplicated, -0.5, 0.0)

# 5. Overall Score Calculation
namematch_distance_max = dwcagent_attr_output['namematch_distance'].max()
similarity = 1.0 - (dwcagent_attr_output['namematch_distance'] / namematch_distance_max) if namematch_distance_max > 0 else 1.0

reliability_subtotal = dwcagent_attr_output[["custom_score_lifetime_data", "custom_score_multiple_names"]].mean(axis=1)
dwcagent_attr_output['custom_score_overall'] = (similarity * reliability_subtotal).round(3)
# print("Overall score calculation completed.")

# 1. Convert numbers to strings (once for the entire column)
score_overall = dwcagent_attr_output["custom_score_overall"].round(2).astype(str)
score_life = dwcagent_attr_output["custom_score_lifetime_data"].round(1).astype(str)
score_multi = dwcagent_attr_output["custom_score_multiple_names"].round(2).astype(str)

# 2. Vectorized merge
dwcagent_attr_output['attributionRemarks'] = (
    dwcagent_attr_output['namematch_distance_annotation'] + "; " +
    score_overall + " (score overall); " +
    dwcagent_attr_output["life_time_periode"] + " (life time); " +
    score_life    + " (life time score); " +
    dwcagent_attr_output["custom_score_lifetime_data_annotation"] + " (life time score note); " +
    score_multi   + " (score multiple names);"
)

# adjust dwcagent displayOrder also to olerall score
dwcagent_attr_output.sort_values(
    # by=['namematch_distance', 'family', 'given', 'custom_score_overall'], 
    # ascending=[True, True, True, False], 
    by=['canonical_string_collector_parsed', 'custom_score_overall', 'family', 'given'], 
    ascending=[True, False, True, True], 
    inplace=True
)
# use ordered canonical_string_collector_parsed to generate displayOrder
temp_duplicated = dwcagent_attr_output['canonical_string_collector_parsed'].duplicated() 
    # duplicated() keeps the first value False and mark all other duplicats as True, i.e. we can cumulate the Trues, it gives the order index
temp_insert_value=temp_duplicated.groupby(dwcagent_attr_output['canonical_string_collector_parsed']).cumsum() + 1 # display order starts at 1, incrementing
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('canonical_string_collector_parsed') + 1, 'displayOrder', temp_insert_value, allow_duplicates=True)

# test an show example data
if explain_and_show_the_data:
    print("example data: names having year of birth (yob) < enventDate_min")
    display(dwcagent_attr_output.loc[dwcagent_attr_output['yob_is_lt_eventDate_min'] == True].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_distance",
        # 'yob', 'yod',
        "life_time_periode", 
        'collectors_eventDate_min', 'collectors_eventDate_max', 
        "yob_is_lt_eventDate_min" ,'yod_is_gt_eventDate_max', 
        'custom_score_lifetime_data', 'custom_score_lifetime_data_annotation'
    ]).head(5))
    print("example data: names having year of birth (yob) > eventDate_min")
    display(dwcagent_attr_output.loc[dwcagent_attr_output['yob_is_lt_eventDate_min'] == False].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_distance",
        # 'yob', 'yod',
        "life_time_periode", 
        'collectors_eventDate_min', 'collectors_eventDate_max', 
        "yob_is_lt_eventDate_min" ,'yod_is_gt_eventDate_max', 
        'custom_score_lifetime_data', 'custom_score_lifetime_data_annotation'
    ]).head(5))


example data: names having year of birth (yob) < enventDate_min


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_distance,life_time_periode,collectors_eventDate_min,collectors_eventDate_max,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime_data,custom_score_lifetime_data_annotation
0,A,Anna Atkins,0.161,0.2696 (k-means distance); 0.16 (score overall...,0.0,0.2696,1799-1871,1949-08-31,2008-03-30,True,False,0.5,"OK yob + 10, but yod not matching, check name ..."
1,A. A. da Silveira,Alvaro Astolpho da Silveira,0.368,0.2001 (k-means distance); 0.37 (score overall...,0.0,0.2001,1867-1945,1897-11-01,1898-06-01,True,True,1.0,full match
2,A. A. du Thouars,Abel Aubert Dupetit Thouars,0.344,0.2364 (k-means distance); 0.34 (score overall...,0.0,0.2364,1793-1864,1829-01-01,1829-01-01,True,True,1.0,full match
3,A. Aaronsohn,Aaron Aaronsohn,0.500,0.0 (k-means distance); 0.5 (score overall); 1...,0.0,0.0000,1876-1919,1908-06-20,1908-06-20,True,True,1.0,full match
5,A. Acebey,Amparo Acebey,0.500,0.0 (k-means distance); 0.5 (score overall); 1...,0.0,0.0000,1973-?,1992-11-28,2000-04-03,True,<NA>,1.0,OK? year of death is missing


example data: names having year of birth (yob) > eventDate_min


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_distance,life_time_periode,collectors_eventDate_min,collectors_eventDate_max,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime_data,custom_score_lifetime_data_annotation
32,A. Andersson,Axel Andersson,-0.500,0.0 (k-means distance); -0.5 (score overall); ...,-0.5,0.0000,1895-?,1891-01-01,1981-08-29,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
34,A. Andersson,I. Anita Andersson,-0.500,0.0 (k-means distance); -0.5 (score overall); ...,-0.5,0.0000,1959-?,1891-01-01,1981-08-29,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
45,A. Araujo-Murakami,Alejandro Araujo-Murakami,-0.250,0.0 (k-means distance); -0.25 (score overall);...,0.0,0.0000,2010-?,2005-03-08,2005-06-10,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
48,A. Artzt,Franz Friedrich Anton Artzt,-0.500,0.0 (k-means distance); -0.5 (score overall); ...,0.0,0.0000,1844-1922,1807-06-01,1927-08-01,False,False,-1.0,life time not matching any eventDate (yob + 10...
52,A. Atmanskich,Joseph Tony Wiskich,-0.063,0.5664 (k-means distance); -0.06 (score overal...,0.0,0.5664,1935-?,1905-08-03,1922-05-08,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."


In [25]:
column_map_dwcagent_attr = {
    'occurrenceID_collectors_firstsample':'occurrenceID',
    'canonical_string_collector_parsed':  'alternateName',
    'source_data':                        'verbatimName',
    'itemLabel':                          'name',
    'item':                               'identifier',
    'collectors_eventDate_min':           'startedAtTime',
    'collectors_eventDate_max':           'endedAtTime',
    'namematch_distance':                 'custom_namematch_distance'
}
dwcagent_attr_output.rename(
    mapper=column_map_dwcagent_attr,
    axis='columns',
    inplace=True)

dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'agentIdentifierType', 'wikidata' , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('agentIdentifierType') + 1, 'agentType'          , 'Person'   , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'action'             , 'collected', allow_duplicates=True)

if explain_and_show_the_data:
    print("the mapped DarwinCore attribution output examples, sorted by alternateName (=collector name) + displayOrder …")
    display(dwcagent_attr_output.head(20))

dwcagent_attr_output=dwcagent_attr_output.reindex(
    columns=[
        'occurrenceID', # no DwC agent standard (yet)?
        'verbatimName',
        'alternateName',
        'displayOrder', # shall start from 1, 2, 3 …
        'name',
        'attributionRemarks',
        'startedAtTime',
        'endedAtTime',
        'agentType',
        'action',
        'agentIdentifierType',
        'identifier',
        "custom_score_overall", # keep it for calculation convenience, no standard in DwC agent
        'custom_namematch_distance',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_multiple_names',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_lifetime_data' # keep it for calculation convenience, no standard in DwC agent
    ]
)
# column deletion not neccessary after ….reindex(columns=[…])
# for this_column in ['yob', 'yod', 'life_time_periode', 'yob_is_lt_eventDate_min', 'yod_is_gt_eventDate_max', 'score_lifetime_annotation']:
#     del dwcagent_attr_output[this_column]

the mapped DarwinCore attribution output examples, sorted by alternateName (=collector name) + displayOrder …


,occurrenceID,alternateName,displayOrder,family,given,custom_namematch_distance,verbatimName,name,identifier,action,...,yod,namematch_distance_annotation,life_time_periode,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime_data,custom_score_lifetime_data_annotation,custom_score_multiple_names,custom_score_overall,attributionRemarks
0,http://id.snsb.info/snsb/collection/462869/564...,A,1,A,NaN,0.2696,"Angeludis C., A.",Anna Atkins,http://www.wikidata.org/entity/Q264269,collected,...,1871,0.2696 (k-means distance),1799-1871,True,False,0.5,"OK yob + 10, but yod not matching, check name ...",0.0,0.161,0.2696 (k-means distance); 0.16 (score overall...
1,https://herbarium.bgbm.org/object/B200129406a,A. A. da Silveira,1,Silveira,A. A. da,0.2001,"Silveira,A.A. da",Alvaro Astolpho da Silveira,http://www.wikidata.org/entity/Q19002102,collected,...,1945,0.2001 (k-means distance),1867-1945,True,True,1.0,full match,0.0,0.368,0.2001 (k-means distance); 0.37 (score overall...
2,https://herbarium.bgbm.org/object/B100365313,A. A. du Thouars,1,Thouars,A. A. du,0.2364,"Thouars,A. A.du Petit-",Abel Aubert Dupetit Thouars,http://www.wikidata.org/entity/Q2821491,collected,...,1864,0.2364 (k-means distance),1793-1864,True,True,1.0,full match,0.0,0.344,0.2364 (k-means distance); 0.34 (score overall...
3,https://herbarium.bgbm.org/object/B100379341,A. Aaronsohn,1,Aaronsohn,A.,0.0000,"Aaronsohn,A.",Aaron Aaronsohn,http://www.wikidata.org/entity/Q2086130,collected,...,1919,0.0 (k-means distance),1876-1919,True,True,1.0,full match,0.0,0.500,0.0 (k-means distance); 0.5 (score overall); 1...
4,https://herbarium.bgbm.org/object/B100217620,A. Abaouz,1,Abaouz,A.,0.5876,"Jury,S.L., Abaouz,A. ,Lafkih,M.Ait & Griffiths...",Jorge A. Abad,http://www.wikidata.org/entity/Q88829543,collected,...,<NA>,0.5876 (k-means distance),?-?,<NA>,<NA>,0.0,unknown life time,0.0,0.000,0.5876 (k-means distance); 0.0 (score overall)...
5,http://id.snsb.info/snsb/collection/719469/786...,A. Acebey,1,Acebey,A.,0.0000,"Acebey, A. (no. 563)",Amparo Acebey,http://www.wikidata.org/entity/Q5673219,collected,...,<NA>,0.0 (k-means distance),1973-?,True,<NA>,1.0,OK? year of death is missing,0.0,0.500,0.0 (k-means distance); 0.5 (score overall); 1...
6,https://herbarium.bgbm.org/object/B100393538,A. Aceres,1,Aceres,A.,0.4445,"Álvarez de Zayas,A., Aceres,A., Bässler,M., Bi...",Giuseppe Acerbi,http://www.wikidata.org/entity/Q55007624,collected,...,1846,0.4445 (k-means distance),1773-1846,True,False,0.5,"OK yob + 10, but yod not matching, check name ...",0.0,0.104,0.4445 (k-means distance); 0.1 (score overall)...
7,https://herbarium.bgbm.org/object/B100092853,A. Ackermann,1,Ackermann,A.,0.0534,"Weigend,M., Ackermann,A. & Castillo,J.A.",Jacob Fidelis Ackermann,http://www.wikidata.org/entity/Q98053,collected,...,1815,0.0534 (k-means distance),1765-1815,True,False,0.5,"OK yob + 10, but yod not matching, check name ...",0.0,0.232,0.0534 (k-means distance); 0.23 (score overall...
8,https://herbarium.bgbm.org/object/B100629441,A. Acosta,1,Acosta,A.,0.0604,"Zardini,E.M. & Acosta,A.",Salvador Acosta Castellanos,http://www.wikidata.org/entity/Q10367096,collected,...,<NA>,0.0604 (k-means distance),1957-?,True,<NA>,1.0,OK? year of death is missing,0.0,0.460,0.0604 (k-means distance); 0.46 (score overall...
9,http://id.snsb.info/snsb/collection/513255/635...,A. Ade,1,Ade,A.,0.0000,"Ade, A.",Alfred Ade,http://www.wikidata.org/entity/Q8195254,collected,...,1968,0.0 (k-means distance),1876-1968,True,True,1.0,full match,0.0,0.500,0.0 (k-means distance); 0.5 (score overall); 1...


In [26]:
if explain_and_show_the_data:
    print("show column-reduced examples of ?multiple name cases …")
    # criterion = dwcagent_attr_output['alternateName'].map(lambda x: x.startswith('S. Ahmad'))
    criterion = dwcagent_attr_output['custom_score_multiple_names'].map(lambda this_score: this_score < 0 )  # show multiple name cases
    
    # display(dwcagent_attr_output[criterion].head(20))
    display(dwcagent_attr_output[criterion].drop(['agentType', 'action', 'agentIdentifierType'], axis='columns').head(20))

show column-reduced examples of ?multiple name cases …


,occurrenceID,verbatimName,alternateName,displayOrder,name,attributionRemarks,startedAtTime,endedAtTime,identifier,custom_score_overall,custom_namematch_distance,custom_score_multiple_names,custom_score_lifetime_data
20,https://herbarium.bgbm.org/object/B100810598,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Oganesian...",A. Aleksanyan,1,Anatolij Aleksandrovič Ničiporovič,0.3904 (k-means distance); 0.0 (score overall)...,2015-07-04,2015-07-09,http://www.wikidata.org/entity/Q26244283,0.000,0.3904,-0.5,0.5
21,https://herbarium.bgbm.org/object/B100810598,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Oganesian...",A. Aleksanyan,2,A.A. Richter,0.3904 (k-means distance); 0.0 (score overall)...,2015-07-04,2015-07-09,http://www.wikidata.org/entity/Q4394909,0.000,0.3904,-0.5,0.5
23,https://herbarium.bgbm.org/object/B101141687,"Miller,A. [Alison]",A. Alison Miller,1,James S. Miller,0.4293 (k-means distance); 0.11 (score overall...,2000-06-30,2000-07-28,http://www.wikidata.org/entity/Q22112339,0.109,0.4293,-0.5,1.0
24,https://herbarium.bgbm.org/object/B101141687,"Miller,A. [Alison]",A. Alison Miller,2,John Frederick Miller,0.4293 (k-means distance); 0.0 (score overall)...,2000-06-30,2000-07-28,http://www.wikidata.org/entity/Q2700645,0.000,0.4293,-0.5,0.5
25,https://herbarium.bgbm.org/object/B101141687,"Miller,A. [Alison]",A. Alison Miller,3,David Miller,0.4293 (k-means distance); 0.0 (score overall)...,2000-06-30,2000-07-28,http://www.wikidata.org/entity/Q5237572,0.000,0.4293,-0.5,0.5
26,https://herbarium.bgbm.org/object/B101141687,"Miller,A. [Alison]",A. Alison Miller,4,Gerrit Smith Miller,0.4293 (k-means distance); 0.0 (score overall)...,2000-06-30,2000-07-28,http://www.wikidata.org/entity/Q538252,0.000,0.4293,-0.5,0.5
33,https://herbarium.bgbm.org/object/B100002495,"Andersson,A.",A. Andersson,1,Karl Alfred Andersson,0.0 (k-means distance); 0.0 (score overall); 1...,1891-01-01,1981-08-29,http://www.wikidata.org/entity/Q131724787,0.000,0.0000,-0.5,0.5
32,https://herbarium.bgbm.org/object/B100002495,"Andersson,A.",A. Andersson,2,Axel Andersson,0.0 (k-means distance); -0.5 (score overall); ...,1891-01-01,1981-08-29,http://www.wikidata.org/entity/Q123652899,-0.500,0.0000,-0.5,-0.5
34,https://herbarium.bgbm.org/object/B100002495,"Andersson,A.",A. Andersson,3,I. Anita Andersson,0.0 (k-means distance); -0.5 (score overall); ...,1891-01-01,1981-08-29,http://www.wikidata.org/entity/Q21505194,-0.500,0.0000,-0.5,-0.5
50,https://herbarium.bgbm.org/object/B101206042,"Asensi,A. & Salvo,E.",A. Asensi,1,Alfredo Asensi,0.0 (k-means distance); 0.25 (score overall); ...,1980-05-01,1980-05-01,http://www.wikidata.org/entity/Q21505377,0.250,0.0000,-0.5,1.0


In [27]:
# TODO further evaluation or filtering, counting, clean up aso.
if not os.path.exists('data'):
    os.makedirs('data')

# this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
# this_timestamp_for_data=20231116
this_output_file='data/results_bgbm_collectors-eventDate_vs_wikidata-botanists_kneighbor_dwc-agent-output_%s.csv' % (
    this_timestamp_for_data
)

dwcagent_attr_output.to_csv(this_output_file, index=False)

print("Wrote matches of collector names as dwc-agent-output into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote matches of collector names as dwc-agent-output into data/results_bgbm_collectors-eventDate_vs_wikidata-botanists_kneighbor_dwc-agent-output_20260210.csv (7430 kB)


## Documentation

Explanation of columns:

Column | Description
-|-
**Botanical collectors** |
family | parsed family name
given | parsed given name
suffix | suffix from name parsing
particle | particle from name parsing
dropping_particle | dropping_particle from name parsing
nick | nick name from name parsing
appellation | appellation from name parsing
title | title from name parsing
eventDate | date of the sampling event (required by GBIF, ☞ https://www.gbif.org/data-quality-requirements-sampling-events)
eventDate_min | calculated earliest date of all the sampling events within the data
eventDate_max | calculated latest date of all the sampling events within the data
eventDate_mean | calculated mean date of all the sampling events within the data
TODO activity_span | Number of years between first and last collection
**Name matching** |
nammatch_collector | matched name of the data set
nammatch_wikidata | matched name; = Wikidata item label name is matched to
namematch_distance | Nearest Neighbour distance between the name and matched name; the lower the value, the better the match
**DarwinCore Agent Output** | (☞ [agent_actions_v2020-09-08.xml](https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml))
occurrenceID | occurrence ID of the data item
name | the interpreted name match (https://github.com/tdwg/attribution/ The name of the item. In this case the *full name* as would be written on a legal document (without abbreviation), eg givenName familyName)
verbatimName | the source data name(s) (https://github.com/tdwg/attribution/ As written on occurrence, such as the collection or determination label.)
alternateName | the input name, collector source name (An alias for the item. Other full name agent may have been known under such as maiden name.)
displayOrder | I guess ordering the multiple name cases (https://github.com/tdwg/attribution/ The display order for the agent that executed the action when more than one agent was a participant.)
attributionRemarks | notes on the results (distance or similarity), including calculated value
agentType | The nature of the agent, e.g. "Person", "Organization", "SoftwareApplication"
action | The name of the single action written as a verb in past tense. Recommended best practice is to use a controlled vocabulary, examples "collected" or "identified"
agentIdentifierType | The type of identifier for the agent. (https://github.com/tdwg/attribution/ Recommended best practice is to use a controlled vocabulary, e.g. “ORCID”, “ISNI”, “Wikidata”, “VIAF”, “RoR”, “Ringgold”, “GRID”).
identifier | Wikidata ID (Recommended practice is to identify the resource by means of a string conforming to an identification system. Examples include International Standard Book Number (ISBN), Digital Object Identifier (DOI), and Uniform Resource Name (URN). Persistent identifiers should be provided as HTTP URIs.)
startedAtTime | (https://github.com/tdwg/attribution/ Start is when an action is deemed to have been started by an agent.) the first date of eventDate (supposedly the first sampling date), but grouped from collector name—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
endedAtTime | (https://github.com/tdwg/attribution/ End is when an action is deemed to have been ended by an agent.) the last date of eventDate (supposedly the last sampling date), but grouped from collector name—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
**Wikidata** |
item | Wikidata Item ID (URL)
itemLabel | Wikidata Item label
surname	| Surname; derived from item label
initials | Initials; derived from item label
canonical_string | Canonical name string; derived from item label, used for matching
orcid | ORCID ([P496](https://www.wikidata.org/wiki/Property:P496))
viaf | VIAF ID ([P214](https://www.wikidata.org/wiki/Property:P214))
isni | ISNI ID ([P213](https://www.wikidata.org/wiki/Property:P496))	
harv | Harvard Index of Botanists ID ([P6264](https://www.wikidata.org/wiki/Property:P6264))
ipni | IPNI author ID ([P586](https://www.wikidata.org/wiki/Property:P586))
abbr | botanist author abbreviation (standard form) ([P428](https://www.wikidata.org/wiki/Property:P428))
bionomia_id | identifier for a collector and/or determiner of natural history specimens, in the Bionomia database ([P6944](https://www.wikidata.org/wiki/Property:P6944))
yob	| Year of birth (derived from [P569](https://www.wikidata.org/wiki/Property:P569))
yod	| Year of death (derived from [P496](https://www.wikidata.org/wiki/Property:P570))
wyb	| Work year period begin ([P2031](https://www.wikidata.org/wiki/Property:P2031))
wye | Work year period end ([P2032](https://www.wikidata.org/wiki/Property:P2032))

Refactoring from <https://github.com/nielsklazenga/avh-collectors/blob/master/match_names_to_wikidata_items.ipynb>

AVH | collector_matching (here)
-|-
avh_matches | collectors_all_matches
wd_test | wd_matchtest